<a href="https://colab.research.google.com/github/LC1332/Luotuo-Chinese-LLM/blob/main/notebook/LAIONclean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 对于MMC4和LAION的清理工作

因为想训一个更好的CLIP，我们不得不在Image Coco Caption之外，去寻找更大的训练集。

这里就不可避免要去考虑MC4和LAION的清理工作

## Source

这里我们可以去考虑Hugging Face上面的LAION 2B

https://huggingface.co/datasets/laion/laion2b-multi-vit-l-14-embeddings/tree/main

而且居然有一个菩萨大神已经求了所有图片的embedding

另一方面是可以去考虑

https://github.com/allenai/mmc4



## 要收集哪些数据

另一方面我们要去考虑的是，要收集哪些数据。其实这里我们希望以CLIP-vit-l-14的词表为例子

如果词表上有一个input已经累积了很多数据，我们就不再考虑增加新的数据了

这样感觉更为合理一些，特别是对于我们初期这种小水管的训练来说

所以这个文档的第一步，我们先来分析VIT-l-14的词表

从这个链接可以看到 vit-l-14 这个模型的用法

https://huggingface.co/openai/clip-vit-large-patch14

让我们先跑上最基础的测试代码

In [ ]:
!pip install transformers
!pip install datasets

In [2]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

# inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

# outputs = model(**inputs)
# logits_per_image = outputs.logits_per_image # this is the image-text similarity score
# probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


当然我们后期可能是真的回去要到完整的CLIP模型的

但是我们在初步我们要的是CLIP的tokenizer

你会发现其实这个tokenizer是存储在processor里面的


In [3]:
print(processor)

CLIPProcessor:
- image_processor: CLIPImageProcessor {
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "feature_extractor_type": "CLIPFeatureExtractor",
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "CLIPImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

- tokenizer: CLIPTokenizerFast(name_or_path='openai/clip-vit-large-patch14', vocab_size=49408, model_max_length=77, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedTok

In [4]:
from transformers import CLIPTokenizer

tokenizer = processor.tokenizer

input = tokenizer.tokenize("a dog is runing")

我们来测试一下这个tokenize的运行时间

已知 某个tokenizer可以使用 input = tokenizer.tokenize( string ) 来进行调用。设计10个英文句子，对这10个句子进行tokenize，并测量平均每次的运行时间

In [5]:
sentences = ["I love Python.", "Python is a great language.", "This is a cat.", 
             "There are many schools in this area.", "The sky is blue.",
             "Mary is a bright student.", "The book is on the table.", 
             "He likes to play football.", "She is a doctor.",
             "The food at this restaurant is delicious."]

import time

total_time = 0
num_sentences = len( sentences )

# 对10个句子进行tokenize，并测量平均每次的运行时间
for sentence in sentences:
  start_time = time.time()
  input = tokenizer.tokenize(sentence)
  end_time = time.time()
  total_time += (end_time - start_time)

avg_time = total_time / num_sentences
print("平均运行时间： %.5f 秒" % avg_time)

平均运行时间： 0.00033 秒


这里我在T4的机器下平均运行时间是  0.00016 秒 ， 让我们来关一下GPU，重新运行一次看看。 是0.00033秒。

看来在tokenizer这一步，是否使用GPU是不要紧的。

因为我们之前已经翻译了coco captioning，让我们来载入coco captioning，来统计一下不同token的出现频率。



In [7]:
from datasets import load_dataset

dataset = load_dataset("ChristophSchuhmann/MS_COCO_2017_URL_TEXT")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/ChristophSchuhmann___parquet/ChristophSchuhmann--MS_COCO_2017_URL_TEXT-01533eba737b92f6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
n_coco =  len( dataset['train']) 

print(n_coco)

591753


In [18]:
for i in range(n_coco):
  text = dataset['train'][i]['TEXT']
  token = tokenizer.tokenize( text )
  print(text)
  print(token)

  if i > 5:
    break

A man with a red helmet on a small moped on a dirt road. 
['a</w>', 'man</w>', 'with</w>', 'a</w>', 'red</w>', 'helmet</w>', 'on</w>', 'a</w>', 'small</w>', 'mo', 'ped</w>', 'on</w>', 'a</w>', 'dirt</w>', 'road</w>', '.</w>']
Man riding a motor bike on a dirt road on the countryside.
['man</w>', 'riding</w>', 'a</w>', 'motor</w>', 'bike</w>', 'on</w>', 'a</w>', 'dirt</w>', 'road</w>', 'on</w>', 'the</w>', 'countryside</w>', '.</w>']
A man riding on the back of a motorcycle.
['a</w>', 'man</w>', 'riding</w>', 'on</w>', 'the</w>', 'back</w>', 'of</w>', 'a</w>', 'motorcycle</w>', '.</w>']
A dirt path with a young person on a motor bike rests to the foreground of a verdant area with a bridge and a background of cloud-wreathed mountains. 
['a</w>', 'dirt</w>', 'path</w>', 'with</w>', 'a</w>', 'young</w>', 'person</w>', 'on</w>', 'a</w>', 'motor</w>', 'bike</w>', 'rests</w>', 'to</w>', 'the</w>', 'foreground</w>', 'of</w>', 'a</w>', 'ver', 'dant</w>', 'area</w>', 'with</w>', 'a</w>', 'bridge

我们看看有没有办法做一个多线程的统计，先取出前1000个元素吧


In [22]:

demo_list = []

for i in range(0,10000):
  text = dataset['train'][i]['TEXT']
  token = tokenizer.tokenize( text )
  demo_list.append( token )

给定一个Python的list of 字符串list，my_strs 其中比如my_strs[0] = ['a</w>', 'man</w>', 'with</w>', 'a</w>', 'red</w>', 'helmet</w>', 'on</w>', 'a</w>', 'small</w>', 'mo', 'ped</w>', 'on</w>', 'a</w>', 'dirt</w>', 'road</w>', '.</w>']

my_strs[1] = ['man</w>', 'riding</w>', 'a</w>', 'motor</w>', 'bike</w>', 'on</w>', 'a</w>', 'dirt</w>', 'road</w>', 'on</w>', 'the</w>', 'countryside</w>', '.</w>']

实现一段python函数，统计my_strs中，不同字符串的出现频率。使用多线程。


In [23]:
from collections import Counter
from concurrent.futures import ThreadPoolExecutor

my_strs = demo_list

def count_frequency(lst):
    return Counter(lst)

with ThreadPoolExecutor(max_workers=16) as executor:
    results = list(executor.map(count_frequency, my_strs))
# 合并所有线程的结果
final_result = sum(results, Counter())
# 打印每个元素及对应的频次

In [26]:
# 根据字典的value值进行排序
sorted_words = sorted(final_result.items(), key=lambda item: item[1], reverse=True)

# 取前10个元素
top_10_words = sorted_words[-10:]

# 打印输出
for i, word in enumerate(top_10_words):
    print(f"{i+1}. {word[0]}: {word[1]}")

1. sai</w>: 1
2. guide</w>: 1
3. entitled</w>: 1
4. ral</w>: 1
5. run: 1
6. lamb</w>: 1
7. snap</w>: 1
8. mad</w>: 1
9. vista</w>: 1
10. stormy</w>: 1


那我们来写一下完整的代码

In [27]:
from datasets import load_dataset
from tqdm.asyncio import tqdm as tqdm

dataset = load_dataset("ChristophSchuhmann/MS_COCO_2017_URL_TEXT")

n_coco =  len( dataset['train']) 

print(n_coco)

demo_list = []

for i in tqdm(range(0,n_coco), total = n_coco, desc = 'move into list'):
  text = dataset['train'][i]['TEXT']
  token = tokenizer.tokenize( text )
  demo_list.append( token )

  0%|          | 0/1 [00:00<?, ?it/s]

591753


move into list: 100%|██████████| 591753/591753 [02:48<00:00, 3516.38it/s]


In [ ]:
from collections import Counter
from concurrent.futures import ThreadPoolExecutor

def count_frequency(lst):
    return Counter(lst)

with ThreadPoolExecutor(max_workers=16) as executor:
    results = list(executor.map(count_frequency, demo_list))
# 合并所有线程的结果
final_result = sum(results, Counter())
# 打印每个元素及对应的频次

In [33]:
print(len(results))
print(results[0])

591753
Counter({'a</w>': 4, 'on</w>': 2, 'man</w>': 1, 'with</w>': 1, 'red</w>': 1, 'helmet</w>': 1, 'small</w>': 1, 'mo': 1, 'ped</w>': 1, 'dirt</w>': 1, 'road</w>': 1, '.</w>': 1})


In [38]:
n = len(results)

batch_size = 1000
n_batch = int(n // batch_size) + 1

sub_counter = []

for i in tqdm(range(0,n_batch), total = n_batch, desc = 'counter in batch'):
  y = min(n-1,(i+1)*batch_size)
  sub_counter.append( sum(results[ i*batch_size : y ], Counter()) )

counter in batch: 100%|██████████| 592/592 [03:29<00:00,  2.83it/s]


In [46]:
coco_train_freq = sum( sub_counter ,  Counter() )

In [52]:
# 根据字典的value值进行排序
sorted_words = sorted(coco_train_freq.items(), key=lambda item: item[1], reverse=True)

print(len(sorted_words), ' words found in coco')

# 取前10个元素
top_10_words = sorted_words[:10]

# 打印输出
for i, word in enumerate(top_10_words):
    print(f"{i+1}. {word[0]}: {word[1]} {word[1]/n_coco * 100}%")

19278  words found in coco
1. a</w>: 978851 165.41546895410755%
2. .</w>: 444432 75.10430872340318%
3. on</w>: 215754 36.46014468874682%
4. of</w>: 204089 34.488883030588774%
5. the</w>: 197831 33.43134720060566%
6. in</w>: 184211 31.12971121396934%
7. with</w>: 154817 26.162436016378454%
8. and</w>: 140931 23.815848842337935%
9. is</w>: 98445 16.636164075213813%
10. man</w>: 73372 12.399092188801747%


我感觉GPT老师给我的不是一个很高效的实现。。不过也先这样吧。。

我做了一个分batch reduce 然后再reduce

跑完了我们来保存一下。。。

In [48]:
import pickle

with open("coco_train_freq.pkl", "wb") as f:
    pickle.dump(coco_train_freq, f)

## 从新数据集上寻找词表中没有的词

接下来让我们试试看载入LAION